## File sorting

The cells below will move data-cube stats tile based results into folders labelled by year. Need to first manually sort the results into discretely labelled folders based on the dcstats query. ie move all the ndvi_max results to a folder called 'ndvi_max', all the ndvi_pct files to a folder called 'ndvi_pct' etc. 

In [10]:
import fnmatch
import os

In [11]:
#create a list of years in format "19901101.tif" so 
#we can aggregate files by year
z = range(1990,2018, 1)
years = []
for zz in z:
    years.append('*' + str(zz) + '1101.tif')

In [6]:
directory='results/mdb_NSW/summer/ndvi_max/'
for file in os.listdir(directory):
    for year in years:
        if not os.path.exists(directory + str(year[1:5])):
            os.mkdir(directory +str(year[1:5])) #create folders of the years
        if fnmatch.fnmatch(file, year): #find matching years and move them to the correct folder
            os.rename(directory +file, directory + year[1:5] + '/' + file)

In [7]:
directory='results/mdb_NSW/summer/ndviArgMaxMin/'
for file in os.listdir(directory):
    for year in years:
        if not os.path.exists(directory + str(year[1:5])):
            os.mkdir(directory +str(year[1:5]))
        if fnmatch.fnmatch(file, year): 
            os.rename(directory +file, directory + year[1:5] + '/' + file)

In [8]:
directory='results/mdb_NSW/summer/ndvi_pct/'
for file in os.listdir(directory):
    for year in years:
        if not os.path.exists(directory + str(year[1:5])):
            os.mkdir(directory +str(year[1:5]))
        if fnmatch.fnmatch(file, year): 
            os.rename(directory +file, directory + year[1:5] + '/' + file)

#### to reverse the above operations

In [ ]:
!find src/dir/ -name '*.tif' -exec mv {} target/dir/ \;

## Build .vrt and mosaic tiff for each year

In [9]:
#build vrt
directory = "/g/data1a/r78/cb3058/dea-notebooks/dcStats/results/mdb_NSW/summer/ndvi_max/"
for folder in os.listdir(directory):
    os.chdir(directory + folder)
    os.system("gdalbuildvrt " + folder + "_mosaic.vrt *.tif")

In [ ]:
#build compressed mosaic
directory = "/g/data1a/r78/cb3058/dea-notebooks/dcStats/results/mdb_NSW/summer/previous_run/ndviArgMaxMin/"
for folder in os.listdir(directory):
    os.chdir(directory + folder)
    os.system("gdal_translate "\
       "-co BIGTIFF=YES "\
       "-co COMPRESS=DEFLATE "\
       "-co ZLEVEL=9 "\
       "-co PREDICTOR=1 "\
       "-co TILED=YES "\
       "-co BLOCKXSIZE=1024 "\
       "-co BLOCKYSIZE=1024 "\
       + folder + "_mosaic.vrt " + folder + "_mosaic.tif")

## Manually do a couple for testing

In [16]:
directory = "/g/data1a/r78/cb3058/dea-notebooks/dcStats/results/mdb_NSW/summer/previous_run/ndvi_max/"
folder='2015'
os.chdir(directory + folder)
!gdal_translate -co BIGTIFF=YES -co COMPRESS=DEFLATE -co ZLEVEL=9 -co PREDICTOR=1 -co TILED=YES -co BLOCKXSIZE=1024 -co BLOCKYSIZE=1024 2015_mosaic.vrt 2015_mosaic.tif

In [11]:
folder='2017'
os.chdir(directory + folder)
!gdal_translate -co BIGTIFF=YES -co COMPRESS=DEFLATE -co ZLEVEL=9 -co PREDICTOR=1 -co TILED=YES -co BLOCKXSIZE=1024 -co BLOCKYSIZE=1024 2017_mosaic.vrt 2017_mosaic.tif

Input file size is 68000, 64000
0...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:
#the argmaxmin ones

In [ ]:
directory = "/g/data1a/r78/cb3058/dea-notebooks/dcStats/results/mdb_NSW/summer/previous_run/ndviArgMaxMin/"
folder='2015'
os.chdir(directory + folder)
!gdal_translate -co BIGTIFF=YES -co COMPRESS=DEFLATE -co ZLEVEL=9 -co PREDICTOR=1 -co TILED=YES -co BLOCKXSIZE=1024 -co BLOCKYSIZE=1024 2015_mosaic.vrt 2015_mosaic.tif

In [ ]:
folder='2017'
os.chdir(directory + folder)
!gdal_translate -co BIGTIFF=YES -co COMPRESS=DEFLATE -co ZLEVEL=9 -co PREDICTOR=1 -co TILED=YES -co BLOCKXSIZE=1024 -co BLOCKYSIZE=1024 2017_mosaic.vrt 2017_mosaic.tif